In [2]:
import IPython

import pandas as pd
import matplotlib.pyplot as plt
import qiime2 as q2
import seaborn as sns

import os

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
PATH = "livia_data_raw/fungut_metadata.tsv"

In [4]:
surveys_df = pd.read_csv(PATH, sep="\t")

In [5]:
!qiime tools peek livia_data_raw/fungut_forward_reads.qza


UUID:        3638611d-1767-413b-9390-70ee3d78e4ff
Type:        SampleData[SequencesWithQuality]
Data format: SingleLanePerSampleSingleEndFastqDirFmt


In [6]:
!qiime demux summarize \
  --i-data livia_data_raw/fungut_forward_reads.qza \
  --o-visualization livia_data_create/demux_summary.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: livia_data_create/demux_summary.qzv


In [ ]:
!qiime dada2 denoise-single \
   --i-demultiplexed-seqs livia_data_raw/fungut_forward_reads.qza \
   --p-trim-left 0 \
   --p-trunc-len 140 \
   --p-min-fold-parent-over-abundance 4 \
   --p-max-ee 4 \
    --o-representative-sequences livia_data_create/rep_seqs.qza \
    --o-table livia_data_create/table.qza \
    --o-denoising-stats livia_data_create/stats.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [ ]:
!qiime feature-table summarize \
  --i-table livia_data_create/table.qza \
  --o-visualization livia_data_create/table_summary.qzv \
--m-sample-metadata-file livia_data_raw/fungut_metadata.tsv